# Projet clustering

Objectif du projet : Utiliser le corpus export_articles_EGC_2004_2018 et faire le bilan de l'évolution de la communauté EGC ces 20 dernières années et tenter d'en prédire l'avenir.

##  Notes :
J'ai abandonné l'utilisation de TreeTagger pour lemmatiser : j'ai pas réussi à le faire fonctionner.

Ainsi que Gensim : la forme de ses corpus est étrange et je pense que c'est une perte de temps à se conformer à leur modèle.

### Imports

In [1]:
import pandas as pd
import numpy as np
import nltk
import string
from nltk.corpus import stopwords
from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer
from random import randint
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from __future__ import print_function
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
from sklearn import metrics
from sklearn.datasets import load_files
from sklearn.cluster import AgglomerativeClustering
import time

In [2]:
start_time = time.clock()

### Ouverture du fichier de données

# Fichier de données original.
doc = pd.read_csv("export_articles_EGC_2004_2018.csv", sep='\t')
doc

In [3]:
# Fichier de données corrigé.
# La correction va jusqu'à la ligne 66.
doc = pd.read_csv("export_articles_EGC_2004_2018_Copie.csv", sep='\t')
doc

,series,booktitle,year,title,abstract,authors,pdf1page,pdfarticle
0,Revue des Nouvelles Technologies de l'Information,EGC,2018,#Idéo2017 : une plateforme citoyenne dédiée à ...,Cette plateforme a pour objectif de permettre ...,"Claudia Marinica, Julien Longhi, Nader Hassine...",http://editions-rnti.fr/render_pdf.php?p1&p=10...,http://editions-rnti.fr/render_pdf.php?p=1002425
1,Revue des Nouvelles Technologies de l'Information,EGC,2018,A two level co-clustering algorithm for very l...,La classification croisée (co-clustering) est ...,"Marius Barctus, Marc Boullé, Fabrice Clérot",http://editions-rnti.fr/render_pdf.php?p1&p=10...,http://editions-rnti.fr/render_pdf.php?p=1002372
2,Revue des Nouvelles Technologies de l'Information,EGC,2018,ALGeoSPF: Un modèle de factorisation basé sur ...,La recommandation de points d'intérêts est dev...,"Jean-Benoît Griesner, Talel Abdesssalem, Huber...",http://editions-rnti.fr/render_pdf.php?p1&p=10...,http://editions-rnti.fr/render_pdf.php?p=1002380
3,Revue des Nouvelles Technologies de l'Information,EGC,2018,Analyse des sentiments à partir des commentair...,L'analyse des sentiments est un processus pend...,"Abdeljalil Elouardighi, Mohcine Maghfour, Hafd...",http://editions-rnti.fr/render_pdf.php?p1&p=10...,http://editions-rnti.fr/render_pdf.php?p=1002397
4,Revue des Nouvelles Technologies de l'Information,EGC,2018,Analyse en rôles sémantiques pour le résumé au...,Cet article présente une approche visant à ext...,"Elyase Lassouli, Yasmine Mesbahi, Camille Prad...",http://editions-rnti.fr/render_pdf.php?p1&p=10...,http://editions-rnti.fr/render_pdf.php?p=1002384
5,Revue des Nouvelles Technologies de l'Information,EGC,2018,Analyse Ontologique de scénario dans un contex...,NaN,"Marwan Batrouni, Aurélie Bertaux, Christophe N...",http://editions-rnti.fr/render_pdf.php?p1&p=10...,http://editions-rnti.fr/render_pdf.php?p=1002414
6,Revue des Nouvelles Technologies de l'Information,EGC,2018,Apport de la fouille de données pour la préven...,Avec plus de 800 000 décès par an dans le mond...,"Romain Billot, Sofian Berrouiguet, Mark Larsen...",http://editions-rnti.fr/render_pdf.php?p1&p=10...,http://editions-rnti.fr/render_pdf.php?p=1002376
7,Revue des Nouvelles Technologies de l'Information,EGC,2018,Apport des modèles locaux pour les K-moyennes ...,"Dans le cadre du clustering prédictif, pour at...","Vincent Lemaire, Oumaima Alaoui Ismaili",http://editions-rnti.fr/render_pdf.php?p1&p=10...,http://editions-rnti.fr/render_pdf.php?p=1002379
8,Revue des Nouvelles Technologies de l'Information,EGC,2018,Apprendre les relations de préférence et de co...,"En classification multi-labels, chaque instanc...","Khalil Laghmari, Christophe Marsala, Mohammed ...",http://editions-rnti.fr/render_pdf.php?p1&p=10...,http://editions-rnti.fr/render_pdf.php?p=1002381
9,Revue des Nouvelles Technologies de l'Information,EGC,2018,Approche contextuelle par régression pour les ...,Les tests A/B sont des procédures utilisées pa...,"Emmanuelle Claeys, Pierre Gançarski, Myriam Ma...",http://editions-rnti.fr/render_pdf.php?p1&p=10...,http://editions-rnti.fr/render_pdf.php?p=1002387


En travaillant avec title et abstract, on va réaliser les transformations.
Tous les documents ont des titres mais ils n'ont pas tous d'abstract.
On va donc utiliser une concaténation du titre et de l'abstract afin d'être sûr d'avoir de l'information.

### Lemmatisation et nettoyage des sujets

Il faut mettre les textes en minuscules, tokeniser, retirer la ponctuation, retirer les stopwords puis lemmatiser.

In [4]:
lemmer = FrenchLefffLemmatizer()

def lemm_tokens(tokens, lemmer):
    lemmed = []
    for item in tokens:
        lemmed.append(lemmer.lemmatize(item))
    return lemmed

def tokenize(text):
    text = text.translate(str.maketrans('','',string.punctuation))
    tokens = nltk.word_tokenize(text)
    lemms = lemm_tokens(tokens,lemmer)
    #stems = stem_tokens(tokens, stemmer)
    return lemms

In [5]:
resume = [tokenize(doc["title"][0].lower())]
abstract = [tokenize(str(doc["abstract"][0].lower()))]
for line in range(len(doc)): # title et abstract ont la même taille.
    if line != 0:
        resume.append(tokenize(doc["title"][line].lower()))
        abstract.append(tokenize(str(doc["abstract"][line]).lower()))

Le traitment des données est acceptable. Un meilleur traitement pourrait être envisagé.

Il faut obtenir un seul jeu de données avec les titres et les abstracts pour pouvoir les traiter et éliminer les cas qui n'ont pas d'abstract.

Il est possible que seul le titre suffise. Il est possible que la racinisation ne soit pas nécéssaire.

In [6]:
for line in range(len(abstract)):
    if abstract[line] != "nan":
        for element in abstract[line]:
            resume[line].append(element)

### Tf-Idf

Attention : l'étape suivante est longue.

In [7]:
dico_time = time.clock()

In [ ]:
dico = []
for line in resume:
    for word in line:
        dico.append(word)
for word in dico:
    if dico.count(word) != 1:
        dico.remove
len(dico)

In [ ]:
print("--- %s seconds ---" % round(time.clock() - dico_time, 2))

In [ ]:
dico = list(set(dico))
len(dico)

In [ ]:
# Calcul du Tf.
tf = []
for x in range(len(resume)):
    tf.append([])
    for y in range(len(resume[x])):
        tf[x].append(resume[x].count(resume[x][y]))

Attention : l'étape suivante est longue.

In [ ]:
df_time = time.clock()

In [ ]:
# Calcul du Df.
df = {}
for x in range(len(dico)):
    sum = 0
    for y in range(len(resume)):
        if dico[x] in resume[y]:
            sum = sum + 1
    df[dico[x]] = sum

In [ ]:
print("--- %s seconds ---" % round(time.clock() - df_time, 2))

In [ ]:
# Calcul de l'Idf
idf = {}
for word in df:
    idf[word] = np.log10(len(resume)/df[word])

In [ ]:
# Calcul du Tf-Idf.
tfidf = []
for x in range(len(resume)):
    tfidf.append({})
    for y in range(len(resume[x])):
        tfidf[x][resume[x][y]] = tf[x][y] * idf[resume[x][y]]
len(tfidf)

Il faut trier les Tf-Idf par valeur et récupérer les 5 meilleurs.

In [ ]:
best = []
for item in tfidf:
    topic = ""
    trier = sorted(item.items(), key=lambda kv: kv[1])
    trier.reverse()
    if len(item)<5:
        top = len(item)
    else:
        top = 5
    for i in range(top):
        topic = topic + trier[i][0] + " "
    best.append(topic)

In [ ]:
best

### Calcul du LSA

Le LSA va calculer, pour un document donné, les documents les plus similaires à son sujet. On classe le résultat le plus similaire dans le cluster du document donné. On reproduit cette étape et on arrête quand à un nombre de clusters ou une similarité trop faible.

En gros, le LSA nous fournit une similarité qu'on utilise pour former une classification hiérarchique.

On aura donc des clusters par sujets. On pourra élaborer nos techniques de datation pour déterminer les tendances et, si possible, créer de nouveaux clusters (avec les K-moyennes) plus précis.

In [ ]:
## vectorisation
# Le tf-idf ne sert à rien d'autre que de convertir notre liste en un vecteur utilisable.
vectorizer = TfidfVectorizer(min_df =1)

dtm = vectorizer.fit_transform(best)

## Document Term Matrix vers Latent Sementic Analysis
lsa =TruncatedSVD(2, algorithm = 'randomized')

dtm_lsa = lsa.fit_transform(dtm)

dtm_lsa = Normalizer(copy=False).fit_transform(dtm_lsa)

In [ ]:
## Calcul de similarité des Documents

#Compute document similarity using LSA components

similarity = np.asarray(np.asmatrix(dtm_lsa) * np.asmatrix(dtm_lsa).T)
# similarity est probablement une matrice

pd.DataFrame(similarity, index=best, columns=best).head(10)

### Clustering Hiérarchique Ascendant (HAC)

Le vrai point d'intérêt de cet algorithme est le seuil de similarité qui servira de point d'arrêt. Pour l'instant, je ne sais pas quel sera ce seuil, il faudra attendre les résultats du clustering.

In [ ]:
clusters = AgglomerativeClustering(n_clusters=10).fit_predict(similarity)

In [ ]:
c0 = []
c1 = []
c2 = []
c3 = []
c4 = []
c5 = []
c6 = []
c7 = []
c8 = []
c9 = []

for i in range(len(clusters)):
    if clusters[i] == 0:
        c0.append({best[i]:doc["year"][i]})
    elif clusters[i] == 1:
        c1.append({best[i]:doc["year"][i]})
    elif clusters[i] == 2:
        c2.append({best[i]:doc["year"][i]})
    elif clusters[i] == 3:
        c3.append({best[i]:doc["year"][i]})
    elif clusters[i] == 4:
        c4.append({best[i]:doc["year"][i]})
    elif clusters[i] == 5:
        c5.append({best[i]:doc["year"][i]})
    elif clusters[i] == 6:
        c6.append({best[i]:doc["year"][i]})
    elif clusters[i] == 7:
        c7.append({best[i]:doc["year"][i]})
    elif clusters[i] == 8:
        c8.append({best[i]:doc["year"][i]})
    elif clusters[i] == 9:
        c9.append({best[i]:doc["year"][i]})

In [ ]:
print(len(c0), len(c1), len(c2), len(c3), len(c4), len(c5), len(c6), len(c7), len(c8), len(c9))

### Travail temporel

In [ ]:
t0 = []
t1 = []
t2 = []
t3 = []
t4 = []
t5 = []
t6 = []
t7 = []
t8 = []
t9 = []

for i in range(len(c0)):
    t0.append(list(c0[i].values()))
for i in range(len(c1)):
    t1.append(list(c1[i].values()))
for i in range(len(c2)):
    t2.append(list(c2[i].values()))
for i in range(len(c3)):
    t3.append(list(c3[i].values()))
for i in range(len(c4)):
    t4.append(list(c4[i].values()))
for i in range(len(c5)):
    t5.append(list(c5[i].values()))
for i in range(len(c6)):
    t6.append(list(c6[i].values()))
for i in range(len(c7)):
    t7.append(list(c7[i].values()))
for i in range(len(c8)):
    t8.append(list(c8[i].values()))
for i in range(len(c9)):
    t9.append(list(c9[i].values()))   

In [ ]:
for i in range(len(c0)):
    t0.append(t0[i][0])
t0 = t0[int(len(t0)/2):]
for i in range(len(c1)):
    t1.append(t1[i][0])
t1 = t1[int(len(t1)/2):]
for i in range(len(c2)):
    t2.append(t2[i][0])
t2 = t2[int(len(t2)/2):]
for i in range(len(c3)):
    t3.append(t3[i][0])
t3 = t3[int(len(t3)/2):]
for i in range(len(c4)):
    t4.append(t4[i][0])
t4 = t4[int(len(t4)/2):]
for i in range(len(c5)):
    t5.append(t5[i][0])
t5 = t5[int(len(t5)/2):]
for i in range(len(c6)):
    t6.append(t6[i][0])
t6 = t6[int(len(t6)/2):]
for i in range(len(c7)):
    t7.append(t7[i][0])
t7 = t7[int(len(t7)/2):]
for i in range(len(c8)):
    t8.append(t8[i][0])
t8 = t8[int(len(t8)/2):]
for i in range(len(c9)):
    t9.append(t9[i][0])
t9 = t9[int(len(t9)/2):]

In [ ]:
T = [t0,t1,t2,t3,t4,t5,t6,t7,t8,t9]

In [ ]:
def minYear(t):
    return np.min(t)

def maxYear(t):
    return np.max(t)

def moyYear(t):
    return int(np.mean(t))

In [ ]:
recent = []
ancien = []
moyen = []
mort = []
nouveau = []

# Si ancien = récent alors récent
# Si récent = ancien alors ancien
# Si moyenne = ancien alors ancien
# Si moyenne = récent alors récent
# Si moyenne = moyen alors moyen

In [ ]:
if minYear(t0) < 2011:
    if maxYear(t0) < 2011:
        mort.append(c0)
    else:
        if moyYear(t0) < 2009:
            ancien.append(c0)
        elif moyYear(t0) > 2014:
            recent.append(c0)
        else:
            moyen.append(c0)
elif minYear(t0) > 2011:
    nouveau.append(c0)
else:
    if moyYear(t0) < 2011:
        ancien.append(c0)
    elif moyYear(t0) > 2011:
        recent.append(c0)
    else:
        moyen.append(c0)
        
if minYear(t1) < 2011:
    if maxYear(t1) < 2011:
        mort.append(c1)
    else:
        if moyYear(t1) < 2009:
            ancien.append(c1)
        elif moyYear(t1) > 2014:
            recent.append(c1)
        else:
            moyen.append(c1)
elif minYear(t1) > 2011:
    nouveau.append(c1)
else:
    if moyYear(t1) < 2011:
        ancien.append(c1)
    elif moyYear(t1) > 2011:
        recent.append(c1)
    else:
        moyen.append(c1)  
        
if minYear(t2) < 2011:
    if maxYear(t2) < 2011:
        mort.append(c2)
    else:
        if moyYear(t2) < 2009:
            ancien.append(c2)
        elif moyYear(t2) > 2014:
            recent.append(c2)
        else:
            moyen.append(c2)
elif minYear(t2) > 2011:
    nouveau.append(c2)
else:
    if moyYear(t2) < 2011:
        ancien.append(c2)
    elif moyYear(t2) > 2011:
        recent.append(c2)
    else:
        moyen.append(c2)
                
if minYear(t3) < 2011:
    if maxYear(t3) < 2011:
        mort.append(c3)
    else:
        if moyYear(t3) < 2009:
            ancien.append(c3)
        elif moyYear(t3) > 2014:
            recent.append(c3)
        else:
            moyen.append(c3)
elif minYear(t3) > 2011:
    nouveau.append(c3)
else:
    if moyYear(t3) < 2011:
        ancien.append(c3)
    elif moyYear(t3) > 2011:
        recent.append(c3)
    else:
        moyen.append(c3)
                
if minYear(t4) < 2011:
    if maxYear(t4) < 2011:
        mort.append(c4)
    else:
        if moyYear(t4) < 2009:
            ancien.append(c4)
        elif moyYear(t4) > 2014:
            recent.append(c4)
        else:
            moyen.append(c4)
elif minYear(t4) > 2011:
    nouveau.append(c4)
else:
    if moyYear(t4) < 2011:
        ancien.append(c4)
    elif moyYear(t4) > 2011:
        recent.append(c4)
    else:
        moyen.append(c4)
                
if minYear(t5) < 2011:
    if maxYear(t5) < 2011:
        mort.append(c5)
    else:
        if moyYear(t5) < 2009:
            ancien.append(c5)
        elif moyYear(t5) > 2014:
            recent.append(c5)
        else:
            moyen.append(c5)
elif minYear(t5) > 2011:
    nouveau.append(c5)
else:
    if moyYear(t5) < 2011:
        ancien.append(c5)
    elif moyYear(t5) > 2011:
        recent.append(c5)
    else:
        moyen.append(c5)
                
if minYear(t6) < 2011:
    if maxYear(t6) < 2011:
        mort.append(c6)
    else:
        if moyYear(t6) < 2009:
            ancien.append(c6)
        elif moyYear(t6) > 2014:
            recent.append(c6)
        else:
            moyen.append(c6)
elif minYear(t6) > 2011:
    nouveau.append(c6)
else:
    if moyYear(t6) < 2011:
        ancien.append(c6)
    elif moyYear(t6) > 2011:
        recent.append(c6)
    else:
        moyen.append(c6)
                
if minYear(t7) < 2011:
    if maxYear(t7) < 2011:
        mort.append(c7)
    else:
        if moyYear(t7) < 2009:
            ancien.append(c7)
        elif moyYear(t7) > 2014:
            recent.append(c7)
        else:
            moyen.append(c7)
elif minYear(t7) > 2011:
    nouveau.append(c7)
else:
    if moyYear(t7) < 2011:
        ancien.append(c7)
    elif moyYear(t7) > 2011:
        recent.append(c7)
    else:
        moyen.append(c7)
                
if minYear(t8) < 2011:
    if maxYear(t8) < 2011:
        mort.append(c8)
    else:
        if moyYear(t8) < 2009:
            ancien.append(c8)
        elif moyYear(t8) > 2014:
            recent.append(c8)
        else:
            moyen.append(c8)
elif minYear(t8) > 2011:
    nouveau.append(c8)
else:
    if moyYear(t8) < 2011:
        ancien.append(c8)
    elif moyYear(t8) > 2011:
        recent.append(c8)
    else:
        moyen.append(c8)
                
if minYear(t9) < 2011:
    if maxYear(t9) < 2011:
        mort.append(c9)
    else:
        if moyYear(t9) <= 2010:
            ancien.append(c9)
        elif moyYear(t9) >= 2013:
            recent.append(c9)
        else:
            moyen.append(c9)
elif minYear(t9) > 2011:
    nouveau.append(c9)
else:
    if moyYear(t9) < 2011:
        ancien.append(c9)
    elif moyYear(t9) > 2011:
        recent.append(c9)
    else:
        moyen.append(c9)

In [ ]:
print("Recent:",recent)
print("Ancien:",ancien)
print("Moyen:",moyen)
print("Mort:",mort)
print("Nouveau:",nouveau)

In [ ]:
print("--- %s seconds ---" % round(time.clock() - start_time, 2))

### Évaluation de la classification

On va maximiser la distance inter cluster et minimiser la distance intra cluster

Distance intra max < Distance inter min

Comparons l'efficacité de la similarité de Jaccard et de la similarité Cosine

In [ ]:
def jaccard(vec1, vec):
    commun = 0
    diff = 0
    for key in vec1.keys():
        if key in vec.keys():
            commun = commun + 1
        else:
            diff = diff + 1
    if diff == 0:
        return -1
    return commun/diff

def cosine(vec1, vec2):
    v1 = np.array(list(vec1.values()))
    v2 = np.array(list(vec2.values()))
    # La partie qui suit devra être retirée plus tard quand on aura des clusters normalisés.
    if len(vec1) > len(vec2):
        v1 = v1[:len(vec2)]
    else:
        v2 = v2[:len(vec1)]
    return np.dot(v1, v2) / (np.sqrt(np.sum(v1**2)) * np.sqrt(np.sum(v2**2)))  

In [ ]:
print("J",jaccard(c0,c1))
print("C",cosine(c0,c1))

In [ ]:
print("--- %s seconds ---" % round(time.clock() - start_time, 2))